[![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/docs/quick-tour/hello-pinecone.ipynb)

# Hello, Pinecone!

This notebook will walk through the steps to get a simple Pinecone index up and running on AWS.


## Prerequisites

Install dependencies.

In [1]:
!pip install -qU \
  pinecone==6.0.1 \
  pandas==2.2.2

## Initializing the Index

Now we need a place to store these embeddings and enable a efficient vector search through them all. To do that we use Pinecone, we can get a [free API key](https://app.pinecone.io/) and enter it below where we will initialize our connection to Pinecone and create a new index.

In [3]:
import os
from getpass import getpass

os.environ["PINECONE_API_KEY"] = os.getenv("PINECONE_API_KEY") or \
    getpass("Enter Pinecone API Key: ")

In [23]:
from pinecone import Pinecone

# configure client
pc = Pinecone()

## Pinecone quickstart

With Pinecone you can create a vector index where you can store and search through your vectors.

In [24]:
# Giving our index a name
index_name = "hello-pinecone"

In [25]:
pc.list_indexes().names()

['hybrid-test', 'index', 'rerankers']

In [26]:
# Delete the index, if an index of the same name already exists
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

We define a `spec` object to define the cloud region we'd like to create an index within. You can find a list of all [available providers and regions here](https://docs.pinecone.io/docs/projects).

In [27]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

Creating a Pinecone Index.

In [28]:
import time

dimensions = 3
pc.create_index(
    name=index_name,
    dimension=dimensions,
    metric="cosine",
    spec=spec
)

# wait for index to be ready before connecting
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

In [29]:
index = pc.Index(name=index_name)

We have the index ready. Now we will create some simple vectors that will serve as our examples.

In [30]:
import pandas as pd

df = pd.DataFrame(
    data={
        "id": ["A", "B"],
        "vector": [[1., 1., 1.], [1., 2., 3.]]
    })
df

id           vector
0  A  [1.0, 1.0, 1.0]
1  B  [1.0, 2.0, 3.0]

We perform upsert operations in our index. This call will insert a new vector in the index or update the vector if the id was already present.

In [31]:
index.upsert(vectors=zip(df.id, df.vector))  # insert vectors

{'upserted_count': 2}

In [32]:
index.describe_index_stats()

{'dimension': 3,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [35]:
index.query(
    vector=[2., 2., 2.],
    top_k=1,
    include_values=True
) # returns top_k matches

{'matches': [{'id': 'A', 'score': 1.0, 'values': [1.0, 1.0, 1.0]}],
 'namespace': '',
 'usage': {'read_units': 6}}

## Delete the Index
Delete the index once you are sure that you do not want to use it anymore. Once the index is deleted, you cannot use it again.

In [36]:
pc.delete_index(name=index_name)

---